# Machine Learning: Data Preparation

**Tian Lou** \
Ohio Education Research Center \
The Ohio State University

**Xiangyu Ren** \
New York University

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.10257277.svg)](https://doi.org/10.5281/zenodo.10257277)

**This notebook is developed for the [Data Literacy and Evidence Building Executive Class](https://www.socialdatascience.umd.edu/data-literacy).**

**The "Syntucky" data, which is synthetic in nature, is exclusively designed for training exercises. It is not intended to derive meaningful insights or make determinations about real-world populations.**

# Goals:

In the previous notebooks, we conducted descriptive analysis to examine job quality for the 2015 Syntucky cohort. We developed several job quality measures, such as earnings, number of jobs, and high-earnings jobs, and investigated how these outcomes vary by degree completion status and major. While descriptive analysis helps us identify important trends in the data, it is hard for us to predict outcomes or evaluate effects of policy interventions by only using descriptive analysis. Therefore, we will introduce you to a more advanced method, machine learning (ML). You will learn how to prepare data, how to run supervised ML models, evaluate and compare different models, and choose the model that is the most suitable for your project. 

We will use two notebooks to illustrate the process of developing and evaluating ML models. In the first notebook, we will focus on **preparing the data**, including generate the label (i.e., outcome or dependent variable) and the features (i.e., predictor or independent variable). In the second notebook, we will **evaluate a hypothetical policy intervention**, i.e., explore whether increasing Pell grant can increase the number of students with high-quality jobs. We will run basic ML models (logistic regression, decision tree, and random forest) and compare the three models by using various metrics (accuracy, precision, and recall). 

**The specific questions we seek to answer in the ML notebooks are**:

1. Which students will have high-earning jobs during the seventh year since their first pursuit of a degree?
2. How important is the Pell grant in determining whether a student has high earnings during the seventh year since their first pursuit of a degree?
3. Which ML model should we use to predict students with high-earning jobs during the seventh year since their first pursuit of a degree?

**After completing this notebook, you should:**
1. Know what a row (unit of analysis) means
2. Understand commonly used ML terms, such as label and features
3. Be able to clean and prepare data for ML models

## 1. Import Data

As usual, we start with importing Python libaries. In addition to the libraries we used in the previous notebooks, such as `pandas` and `numpy`, we also need packages from the `scikit-learn` (or `sklearn`) library in order to prepare the data for ML models. The `scikit-learn` library is a collection of ML tools and allows you to run different types of models, such as regression, classification, clustering, etc. We will import additional packages from `scikit-learn` libary in the second notebook.

In [ ]:
#Data Analysis Libraries
import pandas as pd
import numpy as np

#Machine Learning Packages
from sklearn.preprocessing import MinMaxScaler

Now, let's read in 2013-2015 cohorts' data. We will use 2013 and 2014 cohorts as the training data and 2015 cohort as the testing data. We combine all three cohorts data now so that we can clean the data efficiently. Note that **each row (or our unit of analysis) of the synthetic data represents a student**. Since the synthetic data has already been pre-processed and cleaned, we do not need to worry about data de-duplication here. 

> Before running the code below, please change <font color='red'> **YOUR DATA DIRECTORY**</font> to your own file path.

In [ ]:
#Load 2013, 2014, and 2015 data

#Define data folder directory
data_directory = 'YOUR DATA DIRECTORY'

#Read in different cohort data
df_2013 = pd.read_csv(data_directory+'syntucky_cohort_2013.csv')
df_2014 = pd.read_csv(data_directory+'syntucky_cohort_2014.csv')
df_2015 = pd.read_csv(data_directory+'syntucky_cohort_2015.csv')

#Combine them into one dataset for easier cleaning and generating variables
df_comb = pd.concat([df_2013,df_2014,df_2015])

#Check the first five rows of the combined data
df_comb.head()

## 2. Explore the Trends

Before we dive into the ML models, first, let's use descriptive analysis to investigate the relationship between our key variables: received Pell grant in year 1 and earnings in year 7. [**Pell grant**](https://www2.ed.gov/programs/fpg/index.html) is a federal award for low-income undergraduates who have not earned a bachelor's degree yet. It does not need to be repaid and the award amount depends on students' financial needs, school enrollment status, etc. In the 2022-23 award year, the maximum grant amount is $6,895. *Our hypothesis is that the Pell grant can relieve some students' financial difficulty, help them remain full-time students, and obtain degrees on time. Thus, Pell grant receivers should be more likely to have high-earning jobs than non-receivers in the medium run (such as seven years after their first pursuit of a degree).*

In our data, `first_enroll_acadyr_pell_disbursed` indicates whether a student received non-zero Pell disbursement during the first academic year of degree pursuit. The code below generates summary statistics of year 7 earnings and in-state employment rates for students who received and who did not receive Pell grant in year 1. We can see that while first year Pell grant receivers have a slightly higher in-state employment rate in year 7 than non-receivers, Pell grant receivers' median and mean earnings are a lot lower than that of non-receivers. These findings are the opposite of what we expected. Could it be because Pell grant receivers are low-income students and are more likely to come from disadvantaged demographic groups? Let's see how the results look like when we further break down the data by race groups.

> You may have noticed that some students' Pell grant status has null value. This is because private schools are not required to report Pell grant records. In the later analysis, we will not include these students in the data.

In [ ]:
#Number of people received vs. did not receive Pell grant in year 1
pell_df = df_comb.groupby(['first_enroll_acadyr_pell_disbursed'], dropna = False)['id'].agg(['count']).reset_index()

#Mean and Median year 7 earnings for people received vs. did not receive Pell grant in year 1
y7_earn_pell_df = df_comb.groupby(['first_enroll_acadyr_pell_disbursed'], dropna = False)['year7_earnings'].agg(['mean', 'median', 'count']).reset_index()

#Rename column so that we do not have column name conflicts when merging data
y7_earn_pell_df = y7_earn_pell_df.rename(columns = {'count' : 'y7_emp_count'})

#Merge data
pell_df = pell_df.merge(y7_earn_pell_df, on = 'first_enroll_acadyr_pell_disbursed', how = 'left')

#Calculate percentages
pell_df['y7_emp_pct'] = round(pell_df['y7_emp_count'] / pell_df['count'], 2)

#See results
pell_df

The trends within each race group are very similar to the overall trends. We still see higher in-state employment rates and lower median and mean earnings in year 7 for Pell grant receivers than for non-receivers. There are many other factors we can consider, such as gender compositions and first major choices. However, it's hard for us to consider all these factors at the same time by only using descriptive analysis. Therefore, we use the more advanced method, ML models, to examine how important Pell grant is in determinating year 7 job quality. 

In [ ]:
#Results by Race groups

#Number of people received vs. did not receive Pell grant in year 1 by race group
pell_race_df = df_comb.groupby(['first_enroll_acadyr_pell_disbursed', 'race_group'], dropna = False)['id'].agg(['count']).reset_index()

#Mean and Median year 7 earnings for people received vs. did not receive Pell grant in year 1 by race group
y7_earn_pell_race_df = df_comb.groupby(['first_enroll_acadyr_pell_disbursed', 'race_group'], dropna = False)['year7_earnings'].agg(['mean', 'median', 'count']).reset_index()

#Rename column so that we do not have column name conflicts when merging data
y7_earn_pell_race_df = y7_earn_pell_race_df.rename(columns = {'count' : 'y7_emp_count'})

#Merge data
pell_race_df = pell_race_df.merge(y7_earn_pell_race_df, 
                                  on = ['first_enroll_acadyr_pell_disbursed', 'race_group'], 
                                  how = 'left')

#Calculate percentages
pell_race_df['y7_emp_pct'] = round(pell_race_df['y7_emp_count'] / pell_race_df['count'], 2)

#See results. Here, we also sort the data by race_group. This way, it is easier for us to see within group differences
pell_race_df.sort_values(['race_group'])

## 3. Data Preparation

### 3.1. Create the Label

After we established our research questions, we need to ensure that our data is in the suitable format for ML models. First, we need to construct the label (also called outcome variable, dependent variable, or Y). **We usually use a binary variable as the label and code successful outcomes as 1 and failure as 0.** In this example, we need a label to indicate a student's year 7 job quality. 

- **Job Quality Label**: We developed a few job quality measures in the previous notebooks. Here, we use year 7 earnings to measure job quality. Specifically, we define students who earned above $30,000 during year 7 as having high quality jobs:
  
  - label_high_earnings = 1, if a person earned equal to or above 30,000 dollars in year 7
  - label_high_earnings = 0, if a person earned below 30,000 dollars in year 7
  
  *Note that we will NOT include students who did not have postive year 7 earnings and those whose Pell grant status were null in this model.*

**It is a good practice to check the distribution of your label before running a ML model.** For example, if 99% of students earned more than $30,000 in year 7, then there is no need to use a ML model to predict the outcome. You should consider an alternative measure. In our case, we can see that, among students who had positive earnings in year 7, about 54% of them had high earnings based on our definition. This is a reasonable distribution between success and failure outcomes. We can therefore continue our analysis.

In [ ]:
#Label 1: high earnings in year 7
# = 1 if earned >= $30,000
# = 0 if earned < $30,000 but had positive earnings
#Exclude:
#1. Students who did not have positive earnings in year 7
#2. Students whose Pell grant status were null in year 1
conditions = [((df_comb['year7_earnings'] >= 30000) & (df_comb['first_enroll_acadyr_pell_disbursed'].isnull() == False)),
              ((df_comb['year7_earnings'] < 30000) & (df_comb['year7_earnings'] > 0) & (df_comb['first_enroll_acadyr_pell_disbursed'].isnull() == False))]

choices = [1,
           0]

df_comb['label_high_earnings'] = np.select(conditions, choices, default = np.NaN)

#Check the distribution of the high earnings label
df_high_earnings = df_comb.groupby(['label_high_earnings'])['id'].agg(['count']).reset_index()

df_high_earnings['percent'] = round(df_high_earnings['count'] / df_high_earnings['count'].sum(), 2)

df_high_earnings

#### **Checkpoint 1: Use an Alternative Job Quality Measurement to Create the Label**

In the data measurement notebook, we developed additional job quality measures, including number of jobs (`year7_ct_employers`), employment duration (`year7_ct_qtrs_employed`), and average earnings per employed quater (`year7_earnings` / `year7_ct_qtrs_employed`). Please use one of these measurements or your own measurement to create the job quality label. Also check the distribution of your label.

### 3.2. Create Features

Next, we will create the features for our model (also called predictors, independent variables, or X). We typically have two types of features: categorical and numeric features. Below is the list of features we can use for our ML models:

- **Categorical features**: first_enroll_acadyr_pell_disbursed (first year Pell grant status), gender, race_group, instate_origin, cohort_degree_pursuit_type, first_enroll (first year major)

- **Numeric features**:  birth_year (calculate age at year 7)

You can include other features from the synthetic data if you think they may correlate with job quality. However, **it is important to avoid using information from future as features**. This is because if we use our model to predict the 2015 cohort's outcomes and decide which students should have interventions in year 1, we need to consider what features we have at the point of prediction. For example, one may argue that if a student enrolled in college at year 7, he/she may not have a full-time job or would have a low-wage job, such as a fastfood restaurant job. Indeed, year 7 enrollment may be highly correlated with year 7 earnings. However, if we want to predict a cohort's outcomes at year 1, we would not know if they are going to enroll in college at year 7.

Next, before we transform the features into the formats for ML models, we need to clean our data a little. Specifically, as mentioned in the data measurement notebook, while it is not impossible for someone to earn a doctoral degree within six years of first degree pursuit, we treat these data points as errors and remove them from the data. Also, an extremely small number of students' gender is "Unknown". We will discuss more about how to process missing values later. But since the number of missing values in `gender` is very small, we can just remove them from the data. We also need to generate new features. For example, instead of using `birth_year` directly in the model, we can calculate a student's age at year 7. 

In [ ]:
#Data Cleaning

#Remove doctoral degree recipients due to potential data error
df_comb = df_comb[df_comb['high_completion_label'] != 'Doctoral']

#Check the number of unknown gender
print(df_comb.groupby(['gender'])['id'].agg(['count']).reset_index())

#Remove unknown gender due to small number
df_comb = df_comb[df_comb['gender'] != "Unknown"]

#Generate new features
#Age at year 7
df_comb['year7_age'] = df_comb['cohort_acadyr'] + 6 - df_comb['birth_year']

Since our data has many columns that we will not use as features, labels, or identifiers, we can remove them from the data.

In [ ]:
#Only keep the columns we need

#Identifiers
id_cols = ['id', 'cohort_acadyr']

#Labels
label_cols = ['label_high_earnings']

#Categorical features
cat_cols = ['first_enroll_acadyr_pell_disbursed', 'gender', 'race_group', 'instate_origin', 'first_enroll', 
            'cohort_degree_pursuit_type']

#Numeric features
num_cols = ['year7_age']

#Only keep the features we need
df_comb = df_comb[id_cols + label_cols + cat_cols + num_cols]

#Check the current data
df_comb.head()

Now, we can transform our features into the formats that are suitable for ML models. Specifically, we should:

1. **Avoid dropping missing values**: unless the number of missing values is extremely small, we usually do not drop missing values. Instead, we can generate "missing" indicators for categorical features and impute missing values for numeric features. In the code below, we check the number of null values in each label/feature. Note that not all missing values are null. They could be "", ".", or "Unknown". You should check your features carefully. We can see that the label `label_high_earnings` and the feature `first_enroll_acadyr_pell_disbursed` have null values. It's because we will exclude private school students and those who did not have positive year 7 earnings when using these two variables. So we will just leave them the way they are. Among other features, `first_enroll` and `year7_age` have missing values. We will generate `first_enroll_Missing` to indicate the missingness in first enrolled major. We can fill in the eight missing values in `year7_age` with the average age in year 7 and also bottom code `year7_age` as 18 and top code it as 64.

In [ ]:
#Check number of null values in labels and features
#Note that this method only check if a column has null value. However, missing values could be in many formats.
#You may want to inspect further.
for col in label_cols + cat_cols + num_cols:
    print(col, df_comb[col].isnull().sum())

In [ ]:
#Fill in categorical features' null values with "Missing"
df_comb.loc[df_comb['first_enroll'].isnull() == True, 'first_enroll'] = "Missing"

#Fill in missing age with the average age at year 7. Also bottom and top code age
df_comb.loc[df_comb['year7_age'].isnull() == True, 'year7_age'] = round(df_comb['year7_age'].mean(), 0)

df_comb.loc[df_comb['year7_age'] < 16, 'year7_age'] = 16

df_comb.loc[df_comb['year7_age'] > 64, 'year7_age'] = 64

2. **Convert categorical features into dummies**: we cannot use categorical features directly in the models. Instead, we can create several dummy variables which only contains 0 or 1. For example, for the `race_group` feature, we will create three dummy variables, `race_group_White`, `race_group_Black`, and `race_group_Other`. Of course, when running the model, one of these three dummy variables will be dropped.

   In the code below, we use the `get_dummies` function to generate dummy variables. The function also helps us drop the original categorical features. In the `get_dummies` function, we need to define the columns we want to convert to dummy variables in `columns = `, and the type of the dummy variable's values in `dtype = `. We do not include `first_enroll_acadyr_pell_disbursed` and `instate_origin` in this list because they are already in the 0/1 format.

In [ ]:
#Categorical features that we need to convert to dummy variables
cat_cols = ['gender', 'race_group', 'first_enroll', 'cohort_degree_pursuit_type']

#Get dummy variables
df_comb = pd.get_dummies(df_comb, columns = cat_cols, dtype = float)

#Check our current columns
df_comb.columns

3. **Scale numeric features**: while we can use a numeric feature directly in the model, it is a good practice to standardize the numeric features. This means, regardless the magnitude (hundreds or thousands) or the unit (years or dollars) of the numeric feature, we want to ensure that they are on the same scale. For example, we can scale all numeric features so that their values are between 0 and 1. The reason for scaling is that some distance-based ML algorithms only rely on the numeric values and cannot tell the differences in units. For example, 10-year work experience could be more important than 10,000 dollars prior earnings in determining a person's current earnings. However, in some models, the 10,000 dollars could have a higher weight just because it is a larger number than 10. In the code below, we use the `MinMaxScaler()` function to scale our numeric features to the range [0, 1]. 

In [ ]:
#Define scaler type
scaler = MinMaxScaler()

#Compute the minimum and maximum to be used for scaling
scaler.fit(df_comb['year7_age'].values.reshape(-1,1))

#Scaling features to range [0, 1]
df_comb['year7_age_scl'] = scaler.transform(df_comb['year7_age'].values.reshape(-1,1))

#Drop the original numeric feature
df_comb.drop(columns = ['year7_age'], inplace = True)

Now, we have generated all the labels and features we need. We can do a final check of our data by using the `describe()` function. You should pay attention to the counts and distributions of each column and check whether there are still any missing values.

In [ ]:
#Check summary descriptive statistics for all labels and features
#We use `.T` to tranpose the table so that it is easier to read
#The code in `.apply()` is to format the numbers in the table so that they only have five digits after the decimal
df_comb.describe().T.apply(lambda x: x.apply('{0:.5f}'.format))

#### **Checkpoint 2: Create additional Features**

Try to create additional features. For example, you can add `first_enroll_fulltime` to the feature list or use `urm_status` as a feature instead of `race_group`. Are your new features categorical or numeric? How should you process them? Also, check if your new features have missing values and think about how you should deal with these missing values.

> **Note** that at the beginning of this section (3.2), we only kept the columns we need. Find the "#Only keep the columns we need" cell and rerun all the cells above. Then add the new features you need to the lists, such as `label_cols`, `cat_cols`, and `num_cols` in that cell.

### 3.3. Save the Data

Now we can save the cleaned data and use them for ML models.

> Before running the code below, please change <font color='red'> **YOUR USERNAME**</font> to your username or your own file path.

In [ ]:
df_comb.to_csv(r"C:\Users\YOUR USERNAME\Documents\ML_dataset.csv", index = False)

## Notebook Reference

This notebook is inspired by previous ADA training courses machine learning notebooks, specifically:

Drew. (2019). Coleridge-Initiative/ada-2019-osu: Published version 1 of OSU ADA training course (v1.0.0). Zenodo. https://doi.org/10.5281/zenodo.2672963

Drew. (2019). Coleridge-Initiative/ada-2019-osu: Published version 1 of OSU ADA training course (v1.0.0). Zenodo. https://doi.org/10.5281/zenodo.2672963

Drew. (2019). Coleridge-Initiative/ada-2019-osu: Published version 1 of OSU ADA training course (v1.0.0). Zenodo. https://doi.org/10.5281/zenodo.2672963

Lou, T. (2022, March 18). Machine Learning Model Development and Feature Creation Using Illinois Unemployment Insurance Data. Zenodo. https://doi.org/10.5281/zenodo.6368905

Lou, T. (2022, March 18). Machine Learning Model Deployment and Evaluation Using Illinois Unemployment Insurance Data. Zenodo. https://doi.org/10.5281/zenodo.6369160